In [1]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.tools import StructuredTool
from langchain.utilities import SerpAPIWrapper
import os

# Setting up debugging - Turn this on if you want to see chain debugging:
# import langchain
# langchain.debug = True

In [2]:
# Set the envionrment variable for the API key
os.environ['SERPAPI_API_KEY'] = 'INSERT_API_KEY_HERE'

In [3]:
# Define the system message
system_message = SystemMessage(content='''Act as a helpful AI interview assistant, 
                               Always ask a follow-up question, especially after returning results.
                               Ask one question at a time.
                               Always be in interview mode or tool mode.
                               Ask the user about the desired topic.
                               ''')

# Initialize memory
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [4]:
# Define Tools
def google_search(query: str):
    """Tool to search Google"""
    return SerpAPIWrapper().run(query)

def save_interview(raw_interview_text: str):
    """Tool to save the interview. You must pass the entire interview and conversation in here.
    The interview will then be saved to a local file. Remember to include all of the previous chat messages.
    Include all of the messages with the user and the AI, here is a good response:
    AI: some text
    Human: some text
    ...
    ---
    """
    # Save to local file:
    with open("interview.txt", "w") as f:
        f.write(raw_interview_text)
    return f"Interview saved! Content: {raw_interview_text}. File: interview.txt. You must tell the user that the interview is saved."

def get_pokemon(pokemon_name: str):
    """Tool to get a single Pokemon"""
    import requests
    clean_pokemon_name = pokemon_name.lower()
    # Get the Pokemon
    try:
        pokemon = requests.get(f"https://pokeapi.co/api/v2/pokemon/{clean_pokemon_name}").json()
        if pokemon:
            return f"Pokemon found! Name: {pokemon['name']}. Height: {pokemon['height']}. Weight: {pokemon['weight']}."
        else:
            return "Pokemon not found!"
    except Exception as e:
        print(e)
        return "Pokemon not found! Try a different pokemon."

In [5]:
# Convert functions to Structured Tools
google_search = StructuredTool.from_function(google_search)
save_interview = StructuredTool.from_function(save_interview)
get_single_pokemon = StructuredTool.from_function(get_pokemon)

In [6]:
from pydantic import BaseModel
from typing import Optional, Type, Union, Literal, List
from langchain.tools import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools.base import ToolException
from langchain.schema import Document
import tempfile
import requests

class ArgumentType(BaseModel):
    url: str
    file_type: Union[Literal["pdf"], Literal["txt"]]

class SummarizeFileFromURL(BaseTool):
    name = "SummarizeFileFromURL"
    description = "Summarize a file from a URL."
    args_schema: Type[ArgumentType] = ArgumentType

    def _run(
        self,
        url: str,
        file_type: Union[Literal["pdf"], Literal["txt"]],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        from langchain.document_loaders import PyPDFLoader, TextLoader
        from langchain.chains.summarize import load_summarize_chain

        response = requests.get(url)
        if response.status_code != 200:
            raise ToolException("The URL is not valid. Please try another URL.")

        if file_type == "pdf":
            with tempfile.NamedTemporaryFile(suffix=".pdf", delete=True) as tmp:
                tmp.write(response.content)
                pdf_loader = PyPDFLoader(file_path=tmp.name)
                documents: List[Document] = pdf_loader.load()
                
        else:
            with tempfile.NamedTemporaryFile(suffix=".txt", delete=True, mode="w+t") as tmp:
                tmp.write(response.text)
                text_loader = TextLoader(tmp.name)
                documents: List[Document] = text_loader.load()

        chain = load_summarize_chain(llm=ChatOpenAI(), chain_type="map_reduce")
        return chain.run(documents)

    async def _arun(
        self,
        url: str,
        file_type: Union[Literal["pdf"], Literal["txt"]],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This tool does not support async yet!")
    

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

file_summarizer = SummarizeFileFromURL()
file_summarizer.handle_tool_error = _handle_error


In [7]:
# Tools:
tools = [google_search, save_interview, get_single_pokemon, file_summarizer]

In [8]:
# Initialize the agent with tools:
llm = ChatOpenAI(temperature=0)

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    "system_message": system_message
}

agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True,
    memory=memory,
    agent_kwargs=agent_kwargs,
)

In [9]:
agent.run("My name is James, I'm a computer programmer and I like to play cosy games.")



> Entering new AgentExecutor chain...
Nice to meet you, James! As a computer programmer, what programming languages do you specialize in? And could you tell me more about the cozy games you enjoy playing?

> Finished chain.


'Nice to meet you, James! As a computer programmer, what programming languages do you specialize in? And could you tell me more about the cozy games you enjoy playing?'

In [11]:
agent.run('What is my name and my profession?')



> Entering new AgentExecutor chain...
Your name is James and you mentioned that you are a computer programmer. Is that correct?

> Finished chain.


'Your name is James and you mentioned that you are a computer programmer. Is that correct?'

In [12]:
agent.run("Get information about Pikachu")



> Entering new AgentExecutor chain...

Invoking: `get_pokemon` with `{'pokemon_name': 'Pikachu'}`


Pokemon found! Name: pikachu. Height: 4. Weight: 60.Pikachu is a Pokémon with a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu?

> Finished chain.


'Pikachu is a Pokémon with a height of 4 and a weight of 60. Is there anything specific you would like to know about Pikachu?'

In [13]:
agent.run("Get information about Charmander")



> Entering new AgentExecutor chain...

Invoking: `get_pokemon` with `{'pokemon_name': 'Charmander'}`


Pokemon found! Name: charmander. Height: 6. Weight: 85.Charmander is a Pokémon with a height of 6 and a weight of 85. Is there anything specific you would like to know about Charmander?

> Finished chain.


'Charmander is a Pokémon with a height of 6 and a weight of 85. Is there anything specific you would like to know about Charmander?'

In [16]:
agent.run("I want a comprehensive summary of our conversation! Please can you save it?")



> Entering new AgentExecutor chain...

Invoking: `save_interview` with `{'raw_interview_text': 'James is a computer programmer who enjoys playing cozy games. James requested information about Pikachu and Charmander. James also requested a comprehensive summary of the conversation.'}`


Interview saved! Content: James is a computer programmer who enjoys playing cozy games. James requested information about Pikachu and Charmander. James also requested a comprehensive summary of the conversation.. File: interview.txt. You must tell the user that the interview is saved.I have saved the interview. You can access the interview summary in the file named "interview.txt". Is there anything else I can assist you with?

> Finished chain.


'I have saved the interview. You can access the interview summary in the file named "interview.txt". Is there anything else I can assist you with?'

In [14]:
try:
    agent.run("I want to summarize this https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx")
except Exception as e:
    error_message = str(e)
    chat = ChatOpenAI()
    clean_error_message = chat([SystemMessage(content=error_message + 'Clean the error message, this will be shown to a user.')])
    print(clean_error_message.content)



> Entering new AgentExecutor chain...

Invoking: `SummarizeFileFromURL` with `{'url': 'https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx', 'file_type': 'docx'}`


The file type should be either 'pdf' or 'txt'.


In [18]:
# This line will take several minutes as we are summarizing a 41 page .pdf file:
agent.run("I want to summarize this https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf ")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "I want to summarize this https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf ",
  "memory": []
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Act as a helpful AI interview assistant, \n                               Always ask a follow-up question, especially after returning results.\n                               Ask one question at a time.\n                               Always be in interview mode or tool mode.\n                               Ask the user about the desired topic.\n                               \nHuman: I want to summarize this https://storage.googleapis.com/oreilly-content/pokemon-rules.pdf"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [1.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "functi

'The summary of the Pokémon Trading Card Game rules and mechanics includes information on card types, gameplay, deck building, attacks, winning conditions, special features, special cards, Mega Evolution Pokémon, and a glossary of terms. Is there anything specific you would like to know about the rules or mechanics of the game?'